In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [17]:
## Внимание код работает в версии 1.0
import sklearn
sklearn.__version__

'1.0'

## Ordinal Encoder

In [2]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)
enc.categories_

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [3]:
enc.transform([['Female', 1], ['Male', 1]])

array([[0., 0.],
       [1., 0.]])

In [4]:
enc.categories_

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [5]:
enc.inverse_transform([[1, 0], [0, 1]])

array([['Male', 1],
       ['Female', 2]], dtype=object)

## LabelEncoder

In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit([1, 2, 2, 6])
le.classes_

array([1, 2, 6])

In [7]:
le.transform([1, 1, 2, 6])

array([0, 0, 1, 2])

In [8]:
le.inverse_transform([0, 0, 1, 2])

array([1, 1, 2, 6])

In [9]:
le = preprocessing.LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])
le.classes_

array(['amsterdam', 'paris', 'tokyo'], dtype='<U9')

In [10]:
le.transform(["tokyo", "tokyo", "paris"])

array([2, 2, 1])

In [11]:
le.inverse_transform([2, 2, 1])

array(['tokyo', 'tokyo', 'paris'], dtype='<U9')

## OneHotEncoder

In [12]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)
enc.categories_

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [13]:
# handle_unknown='ignore' ВНИМАНИЕ неивестные обращаются в 0-вектор
enc.transform([['Female', 1], ['Male', 4]]).toarray()

array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [14]:
enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

array([['Male', 1],
       [None, 2]], dtype=object)

In [18]:
enc.get_feature_names_out(['gender', 'group'])

array(['gender_Female', 'gender_Male', 'group_1', 'group_2', 'group_3'],
      dtype=object)

In [19]:
drop_enc = OneHotEncoder(drop='first').fit(X)
drop_enc.categories_

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [44]:
drop_enc.transform([['Female', 1], ['Male', 2]]).toarray()

array([[0., 0., 0.],
       [1., 1., 0.]])

In [41]:
drop_binary_enc = OneHotEncoder(drop='if_binary').fit(X)
drop_binary_enc.transform([['Female', 1], ['Male', 2],['Female', 1]]).toarray()

array([[0., 1., 0., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.]])

## Преобразование колонок различных типов

In [46]:
import pandas as pd
X = pd.DataFrame(
    {'city': ['London', 'London', 'Paris', 'Sallisaw'],
     'title': ["His Last Bow", "How Watson Learned the Trick",
               "A Moveable Feast", "The Grapes of Wrath"],
     'expert_rating': [5, 3, 4, 5],
     'user_rating': [4, 5, 4, 3]})
X

,city,title,expert_rating,user_rating
0,London,His Last Bow,5,4
1,London,How Watson Learned the Trick,3,5
2,Paris,A Moveable Feast,4,4
3,Sallisaw,The Grapes of Wrath,5,3


In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
ct = ColumnTransformer([
      ('scale', StandardScaler(),
      make_column_selector(dtype_include=np.number)),
      ('onehot',
      OneHotEncoder(),
      make_column_selector(pattern='city', dtype_include=object))])

ct.fit_transform(X)

array([[ 0.90453403,  0.        ,  1.        ,  0.        ,  0.        ],
       [-1.50755672,  1.41421356,  1.        ,  0.        ,  0.        ],
       [-0.30151134,  0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.90453403, -1.41421356,  0.        ,  0.        ,  1.        ]])

In [51]:
ct.get_feature_names_out()

array(['scale__expert_rating', 'scale__user_rating',
       'onehot__city_London', 'onehot__city_Paris',
       'onehot__city_Sallisaw'], dtype=object)

In [64]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector

display(X)

ct_r = ColumnTransformer([
      ('scale', StandardScaler(),
      make_column_selector(pattern='expert_rating')),
      ('onehot',
      OneHotEncoder(),
      make_column_selector(pattern='city', dtype_include=object)),
      ('drop_colunm',
       'drop',
      make_column_selector(pattern='title', dtype_include=object))
      ],
      remainder='passthrough')

X_cl = ct_r.fit_transform(X)

pd.DataFrame(X_cl, columns = ct_r.get_feature_names_out()) 

,city,title,expert_rating,user_rating
0,London,His Last Bow,5,4
1,London,How Watson Learned the Trick,3,5
2,Paris,A Moveable Feast,4,4
3,Sallisaw,The Grapes of Wrath,5,3


,scale__expert_rating,onehot__city_London,onehot__city_Paris,onehot__city_Sallisaw,remainder__user_rating
0,0.904534,1.0,0.0,0.0,4.0
1,-1.507557,1.0,0.0,0.0,5.0
2,-0.301511,0.0,1.0,0.0,4.0
3,0.904534,0.0,0.0,1.0,3.0


In [72]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler

display(X)

column_trans = make_column_transformer(
    (OneHotEncoder(), ['city']),
    ('drop', 'title'),
    remainder=MinMaxScaler())
column_trans

,city,title,expert_rating,user_rating
0,London,His Last Bow,5,4
1,London,How Watson Learned the Trick,3,5
2,Paris,A Moveable Feast,4,4
3,Sallisaw,The Grapes of Wrath,5,3


ColumnTransformer(remainder=MinMaxScaler(),
                  transformers=[('onehotencoder', OneHotEncoder(), ['city']),
                                ('drop', 'drop', 'title')])

In [71]:
X_col_tr = column_trans.fit_transform(X)
pd.DataFrame(X_col_tr , columns = column_trans.get_feature_names_out()) 

,onehotencoder__city_London,onehotencoder__city_Paris,onehotencoder__city_Sallisaw,remainder__expert_rating,remainder__user_rating
0,1.0,0.0,0.0,1.0,0.5
1,1.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.5,0.5
3,0.0,0.0,1.0,1.0,0.0
